# Predicting Data Scientists Salary in India

Data scientist is the sexiest job in the world. How many times have you heard that? Analytics India Annual Salary Study which aims to understand a wide range of trends data science says that the median analytics salary in India for the year 2017 is INR 12.7 Lakhs across all experience level and skill sets. So given the job description and other key information can you predict the range of salary of the job posting? What kind of factors influence the salary of a data scientist? The study also says that in the world of analytics, Mumbai is the highest paymaster at almost 13.3 Lakhs per annum, followed by Bengaluru at 12.5 Lakhs. The industry of the data scientist can also influence the salary. Telecom industry pays the highest median salaries to its analytics professionals at 18.6 Lakhs. What are you waiting for, solve the problem by predicting how much a data scientist or analytics professional will be paid by analysing the data given. Bonus Tip: You can analyse the data and get key insights for your career as well. The best data scientists and machine learning engineers will be given awesome prizes at the end of hackathon. Share this hackathon with a colleague who may be interested in mining the dataset for insights and make great predictions. Data The dataset is based on salary and job postings in India across the internet. The train and the test data consists of attributes mentioned below. The rows of train dataset has rich amount of information regarding the job posting such as name of the designation and key skills required for the job. The training data and test data comprise of 19802 samples and of 6601 samples each. This is a dataset which has been collected over some time to gather relevant analytics jobs posting over the years. Features Name of the company (Encoded) Years of experience Job description Job designation Job Type Key skills Location Salary in Rupees Lakhs(To be predicted) Problem Statement Based on the given attributes and salary information, build a robust machine learning model that predicts the salary range of the salary post.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier,StackingClassifier

from sklearn.metrics import classification_report,confusion_matrix,cohen_kappa_score
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

## import dataset

In [2]:
train = pd.read_csv('Final_Train_Dataset.csv')
test = pd.read_csv('Final_Test_Dataset.csv')

In [3]:
train.head()

,Unnamed: 0,experience,job_description,job_desig,job_type,key_skills,location,salary,company_name_encoded
0,0,5-7 yrs,Exp: Minimum 5 years;Good understanding of IOC...,Senior Exploit and Vulnerability Researcher,NaN,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri),6to10,3687
1,1,10-17 yrs,He should have handled a team of atleast 5-6 d...,Head SCM,NaN,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458
2,2,5-9 yrs,Must be an effective communicator (written & s...,Deputy Manager - Talent Management & Leadershi...,Analytics,"HR Analytics, Employee Engagement, Training, S...",Delhi NCR,15to25,4195
3,3,7-10 yrs,7 - 10 years of overall experience in data e...,Associate Manager Data Engineering,Analytics,"SQL, Javascript, Automation, Python, Ruby, Ana...",Bengaluru,10to15,313
4,4,1-3 yrs,Chartered Accountancy degree or MBA in Finance...,TS- GSA- Senior Analyst,NaN,"accounting, finance, cash flow, financial plan...",Gurgaon,3to6,1305


In [4]:
test.head()

,experience,job_description,job_desig,job_type,key_skills,location,company_name_encoded
0,7-12 yrs,Professional experience in Java/J2EE based ser...,IT Technology Senior Consultant/java/ J2ee/ Se...,NaN,"Java, J2Ee, Tomcat, JBoss, Weblogic, Oracle, E...",Bengaluru,2066
1,0-5 yrs,We are looking for 20+ Fresher/Experienced Can...,Medical Billing Process | International KPO | ...,NaN,"Medical Billing, Insurance Processing",Ahmedabad(Sola),2629
2,3-6 yrs,Should understand overall integration framewor...,Oracle Sales Cloud Functional Consultant,NaN,"Oracle Sales, Functional Consultancy, Troubles...",Bengaluru,2448
3,0-3 yrs,NaN,Looking For Freshers WHO WANT To Work WITH US,NaN,"offline, online, internet, part time, home bas...","Delhi NCR, Chennai, Hyderabad, Gurgaon, Luckno...",2711
4,0-5 yrs,NaN,Process Associate / Sr Process Associate / Tec...,NaN,"voice support, analytical skills, Process asso...",Hyderabad,40


In [5]:
train.shape

(19802, 9)

In [6]:
test.shape

(6601, 7)

In [7]:
train.nunique()

Unnamed: 0              19802
experience                129
job_description          9059
job_desig               11708
job_type                    5
key_skills              12951
location                 1504
salary                      6
company_name_encoded     5035
dtype: int64

In [8]:
test.nunique()

experience               110
job_description         4064
job_desig               5135
job_type                   5
key_skills              5589
location                 803
company_name_encoded    2752
dtype: int64

In [9]:
train.isna().sum()

Unnamed: 0                  0
experience                  0
job_description          4418
job_desig                   0
job_type                15005
key_skills                  1
location                    0
salary                      0
company_name_encoded        0
dtype: int64

# Data Pre-Processing

In [10]:
train = train.dropna(subset=['key_skills'])

df_train = train[['key_skills', 'job_desig', 'job_description', 'location', 'job_type', 'experience','salary']]
df_test = test[['key_skills', 'job_desig', 'job_description', 'job_type', 'experience', 'location']]

In [11]:
import re

def clean_skills(skl):
    skills = str(skl).lower()
    skills = re.sub('\...','',skills)
    skills = re.sub(',','',skills)
    skills = re.sub(r'\s+', ' ', skills)
    return skills

df_train['skills_cleaned'] = df_train['key_skills'].apply(clean_skills)
df_test['skills_cleaned'] = df_test['key_skills'].apply(clean_skills)

In [12]:
df_train.head()

,key_skills,job_desig,job_description,location,job_type,experience,salary,skills_cleaned
0,"team skills, communication skills, analytical ...",Senior Exploit and Vulnerability Researcher,Exp: Minimum 5 years;Good understanding of IOC...,Delhi NCR(Vikas Puri),NaN,5-7 yrs,6to10,team skills communication skills analytical sk...
1,"ppc, logistics, inventory management, supply c...",Head SCM,He should have handled a team of atleast 5-6 d...,Sonepat,NaN,10-17 yrs,10to15,ppc logistics inventory management supply chai...
2,"HR Analytics, Employee Engagement, Training, S...",Deputy Manager - Talent Management & Leadershi...,Must be an effective communicator (written & s...,Delhi NCR,Analytics,5-9 yrs,15to25,hr analytics employee engagement training succ...
3,"SQL, Javascript, Automation, Python, Ruby, Ana...",Associate Manager Data Engineering,7 - 10 years of overall experience in data e...,Bengaluru,Analytics,7-10 yrs,10to15,sql javascript automation python ruby analytic...
4,"accounting, finance, cash flow, financial plan...",TS- GSA- Senior Analyst,Chartered Accountancy degree or MBA in Finance...,Gurgaon,NaN,1-3 yrs,3to6,accounting finance cash flow financial plannin...


In [13]:
train.job_description.fillna('missing',inplace = True)
test['job_description'].fillna('missing', inplace=True)

def clean_job_desc(job):
    job_desc = str(job).lower()
    job_desc = re.sub(r'[^a-z]', ' ', job_desc)
    job_desc = re.sub(r'\s+', ' ', job_desc)
    return job_desc

df_train['job_desc_cleaned'] = df_train['job_description'].apply(clean_job_desc)
df_test['job_desc_cleaned'] = df_test['job_description'].apply(clean_job_desc)

In [14]:
def clean_location(loc):
    location = loc.lower()
    location = re.sub(r'[^a-z]', ' ', location)
    location = re.sub(r'\s+', ' ', location)
    return location

df_train['loc_cleaned'] = df_train['location'].apply(clean_location)
df_test['loc_cleaned'] = df_test['location'].apply(clean_location)

In [15]:
train['job_type'].fillna('missingjobtype', inplace=True)
train['job_type'].replace('Analytics', 'analytics', inplace=True)
train['job_type'].replace('Analytic', 'analytics', inplace=True)
train['job_type'].replace('ANALYTICS', 'analytics', inplace=True)
train['job_type'].replace('analytic', 'analytics', inplace=True)

test['job_type'].fillna('missingjobtype', inplace=True)
test['job_type'].replace('Analytics', 'analytics', inplace=True)
test['job_type'].replace('Analytic', 'analytics', inplace=True)
test['job_type'].replace('ANALYTICS', 'analytics', inplace=True)
test['job_type'].replace('analytic', 'analytics', inplace=True)

df_train['job_type_cleaned'] = train['job_type'] 
df_test['job_type_cleaned'] = test['job_type']

In [16]:
df_train.isna().sum()

key_skills              0
job_desig               0
job_description      4417
location                0
job_type            15005
experience              0
salary                  0
skills_cleaned          0
job_desc_cleaned        0
loc_cleaned             0
job_type_cleaned        0
dtype: int64

In [17]:
def min_exp(val):
    exp = re.sub('-',' ',val)
    exp = exp.split(" ")
    exp = int(exp[0])
    return exp
    
def max_exp(val):
    exp = re.sub('-',' ',val)
    exp = exp.split(' ')
    exp = int(exp[1])
    return exp
    
df_train['min_exp'] = df_train['experience'].apply(lambda x : min_exp(x))
df_train['max_exp'] = df_train['experience'].apply(lambda x : max_exp(x))

df_test['min_exp'] = df_test['experience'].apply(lambda x : min_exp(x))
df_test['max_exp'] = df_test['experience'].apply(lambda x : max_exp(x))
        

In [18]:
def clean_job_desig(desig):
    job_desig = desig.lower()
    job_desig = re.sub(r'[^a-z]', ' ', job_desig)
    job_desig = re.sub(r'\s+', ' ', job_desig)
    return job_desig

df_train['desig_cleaned'] = df_train['job_desig'].apply(clean_job_desig)
df_test['desig_cleaned'] = df_test['job_desig'].apply(clean_job_desig)

In [19]:
df_train['merged'] = (df_train['desig_cleaned'] + ' ' + df_train['job_desc_cleaned'] + ' ' + df_train['skills_cleaned']
                      + ' ' + df_train['job_type_cleaned'])

df_test['merged'] = (df_test['desig_cleaned'] + ' ' + df_test['job_desc_cleaned'] + ' ' + df_test['skills_cleaned']
                     + ' ' + df_test['job_type_cleaned'])

In [20]:
data_train  = df_train[['merged', 'loc_cleaned', 'min_exp', 'max_exp']] 
data_test = df_test[['merged', 'loc_cleaned', 'min_exp', 'max_exp']] 

In [21]:
data_train = data_train.rename(columns = {'merged':'emp_info'},inplace = False)
data_test = data_test.rename(columns = {'merged':'emp_info'},inplace = False)

In [22]:
def min_sal(sal):
    val = str(sal).split("to")
    return val[0]
def max_sal(sal):
    val = str(sal).split("to")
    return val[1]

target = pd.DataFrame()
target["min_sal"] = df_train["salary"].apply(lambda x: min_sal(x))
target["max_sal"] = df_train["salary"].apply(lambda x: max_sal(x))
target1 = target.min_sal
target2 = target.max_sal

In [23]:
target.head()

,min_sal,max_sal
0,6,10
1,10,15
2,15,25
3,10,15
4,3,6


# Encoding

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['salary'] = le.fit_transform(train['salary'])

# Train-Test Split

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(
    data_train,train['salary'], test_size=0.20, 
    stratify=train['salary'], random_state=75)

In [26]:
print('No. of sample texts X_train: ', len(X_train))
print('No. of sample texts X_cv   : ', len(X_cv))


No. of sample texts X_train:  15840
No. of sample texts X_cv   :  3961


# Predictive Modelling

In [27]:
X_train_merged = X_train['emp_info']
X_train_loc = X_train['loc_cleaned']

X_cv_merged = X_cv['emp_info']
X_cv_loc = X_cv['loc_cleaned']

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tf1 = TfidfVectorizer(min_df=3, token_pattern=r'\w{3,}', ngram_range=(1,3), max_df=0.9)
tf2 = TfidfVectorizer(min_df=2, token_pattern=r'\w{3,}')

X_train_merged = tf1.fit_transform(X_train_merged)
X_train_loc = tf2.fit_transform(X_train_loc)

X_cv_merged = tf1.transform(X_cv_merged)
X_cv_loc = tf2.transform(X_cv_loc)

In [29]:
from scipy import sparse
from sklearn.preprocessing import StandardScaler

sc1 = StandardScaler()
X_train_MinExp = sc1.fit_transform(np.array(X_train['min_exp']).reshape(-1,1))
X_cv_MinExp = sc1.transform(np.array(X_cv['min_exp']).reshape(-1,1))
X_train_MinExp = sparse.csr_matrix(X_train_MinExp)
X_cv_MinExp = sparse.csr_matrix(X_cv_MinExp)

sc2 = StandardScaler()
X_train_MaxExp = sc2.fit_transform(np.array(X_train['max_exp']).reshape(-1,1))
X_cv_MaxExp = sc2.transform(np.array(X_cv['max_exp']).reshape(-1,1))
X_train_MaxExp = sparse.csr_matrix(X_train_MaxExp)
X_cv_MaxExp = sparse.csr_matrix(X_cv_MaxExp)

In [30]:
from scipy.sparse import hstack, csr_matrix

merged_train = hstack((X_train_merged, X_train_loc, X_train_MinExp, X_train_MaxExp))
merged_cv  = hstack((X_cv_merged, X_cv_loc, X_cv_MinExp, X_cv_MaxExp))

In [31]:
merged_train.shape, merged_cv.shape

((15840, 52320), (3961, 52320))

In [32]:
mod = []
accu = []
prec = []
rec = []
f1 = []
kappa = []

def model_validation(model, xtrain, ytrain, xtest, ytest):
    m = model
    m.fit(xtrain, ytrain)
    pred_h = m.predict(xtest)
    
    # Print evaluation metrics
    print('Confusion Matrix\n', confusion_matrix(ytest, pred_h))
    print('Classification Report\n', classification_report(ytest, pred_h))
    
    # Print other metrics
    print(f'Accuracy: {accuracy_score(ytest, pred_h):.4f}')
    print(f'Recall: {recall_score(ytest, pred_h, average="weighted"):.4f}')
    print(f'Precision: {precision_score(ytest, pred_h, average="weighted"):.4f}')
    print(f'F1 Score: {f1_score(ytest, pred_h, average="weighted"):.4f}')
    print(f'Cohen Kappa: {cohen_kappa_score(ytest, pred_h):.4f}')
    
    # Save model evaluation metrics if needed
    response = input('Do you want to save a model Y/N? ')
    
    if response.lower() == 'y':
        global scorecard
        mod.append(str(model))
        accu.append(accuracy_score(ytest, pred_h))
        rec.append(recall_score(ytest, pred_h, average='weighted'))
        prec.append(precision_score(ytest, pred_h, average='weighted'))
        f1.append(f1_score(ytest, pred_h, average='weighted'))
        kappa.append(cohen_kappa_score(ytest, pred_h))

        scorecard = pd.DataFrame({'Model': mod,
                                 'Accuracy': accu,
                                 'Precision': prec,
                                 'Recall': rec,
                                 'F1 Score': f1,
                                 'Cohen Kappa': kappa})
        print(scorecard)
    else:
        return

In [33]:
model = LogisticRegression()
model_validation(model, merged_train, y_train, merged_cv, y_cv)

Confusion Matrix
 [[499  30   3   0  86  31]
 [ 25 368 263   5  32 207]
 [  7 313 356  71  10  68]
 [  5  26 148 132   0   4]
 [190  70  18   0 149 138]
 [ 58 249  89   8 122 181]]
Classification Report
               precision    recall  f1-score   support

           0       0.64      0.77      0.70       649
           1       0.35      0.41      0.38       900
           2       0.41      0.43      0.42       825
           3       0.61      0.42      0.50       315
           4       0.37      0.26      0.31       565
           5       0.29      0.26      0.27       707

    accuracy                           0.43      3961
   macro avg       0.44      0.42      0.43      3961
weighted avg       0.42      0.43      0.42      3961

Accuracy: 0.4254
Recall: 0.4254
Precision: 0.4212
F1 Score: 0.4187
Cohen Kappa: 0.2940
Do you want to save a model Y/N? y
                  Model  Accuracy  Precision    Recall  F1 Score  Cohen Kappa
0  LogisticRegression()  0.425398   0.421242  0.42539

In [34]:
model_validation(DecisionTreeClassifier(max_depth=5),merged_train, y_train, merged_cv, y_cv)

Confusion Matrix
 [[535  52   7   0  23  32]
 [ 88 471 228  28   7  78]
 [ 47 247 406 102   4  19]
 [  6  44 118 142   0   5]
 [214 128  49   0  67 107]
 [119 257 116  14  40 161]]
Classification Report
               precision    recall  f1-score   support

           0       0.53      0.82      0.65       649
           1       0.39      0.52      0.45       900
           2       0.44      0.49      0.46       825
           3       0.50      0.45      0.47       315
           4       0.48      0.12      0.19       565
           5       0.40      0.23      0.29       707

    accuracy                           0.45      3961
   macro avg       0.46      0.44      0.42      3961
weighted avg       0.45      0.45      0.42      3961

Accuracy: 0.4499
Recall: 0.4499
Precision: 0.4464
F1 Score: 0.4209
Cohen Kappa: 0.3225
Do you want to save a model Y/N? y
                                 Model  Accuracy  Precision    Recall  \
0                 LogisticRegression()  0.425398   0.42124

In [35]:
model = RandomForestClassifier(n_estimators=150,max_features=11,max_depth=5)
model_validation(model, merged_train, y_train, merged_cv, y_cv)

Confusion Matrix
 [[  0 649   0   0   0   0]
 [  0 894   6   0   0   0]
 [  0 818   7   0   0   0]
 [  0 314   1   0   0   0]
 [  0 565   0   0   0   0]
 [  0 703   4   0   0   0]]
Classification Report
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       649
           1       0.23      0.99      0.37       900
           2       0.39      0.01      0.02       825
           3       0.00      0.00      0.00       315
           4       0.00      0.00      0.00       565
           5       0.00      0.00      0.00       707

    accuracy                           0.23      3961
   macro avg       0.10      0.17      0.06      3961
weighted avg       0.13      0.23      0.09      3961

Accuracy: 0.2275
Recall: 0.2275
Precision: 0.1325
F1 Score: 0.0873
Cohen Kappa: 0.0004
Do you want to save a model Y/N? y
                                               Model  Accuracy  Precision  \
0                               LogisticRegression()  

In [36]:
model = AdaBoostClassifier(n_estimators=150,learning_rate=0.1)
model_validation(model, merged_train, y_train, merged_cv, y_cv)

Confusion Matrix
 [[478  39   3   0  69  60]
 [ 21 387 251   1  44 196]
 [  9 277 459   0  15  65]
 [  3  31 272   2   1   6]
 [166 102  11   1  97 188]
 [ 53 240  69   0  92 253]]
Classification Report
               precision    recall  f1-score   support

           0       0.65      0.74      0.69       649
           1       0.36      0.43      0.39       900
           2       0.43      0.56      0.49       825
           3       0.50      0.01      0.01       315
           4       0.31      0.17      0.22       565
           5       0.33      0.36      0.34       707

    accuracy                           0.42      3961
   macro avg       0.43      0.38      0.36      3961
weighted avg       0.42      0.42      0.40      3961

Accuracy: 0.4231
Recall: 0.4231
Precision: 0.4208
F1 Score: 0.3973
Cohen Kappa: 0.2842
Do you want to save a model Y/N? y
                                               Model  Accuracy  Precision  \
0                               LogisticRegression()  

In [37]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [38]:
import lightgbm as lgb

In [39]:
lgb_model = lgb.LGBMClassifier()

model_validation(lgb_model, merged_train, y_train, merged_cv, y_cv)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.146417 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 110290
[LightGBM] [Info] Number of data points in the train set: 15840, number of used features: 4432
[LightGBM] [Info] Start training from score -1.808567
[LightGBM] [Info] Start training from score -1.481605
[LightGBM] [Info] Start training from score -1.568616
[LightGBM] [Info] Start training from score -2.531427
[LightGBM] [Info] Start training from score -1.947616
[LightGBM] [Info] Start training from score -1.724030
Confusion Matrix
 [[476  20   7   0 100  46]
 [ 23 334 308   7  41 187]
 [  8 267 369  99  16  66]
 [  3  31 135 142   1   3]
 [179  69  11   1 165 140]
 [ 54 232 115   8 126 172]]
Classification Report
               precision    recall  f1-score   support

           0       0.64      0.73      0.68       649
      

Decision Tree Classifier has best results hence we will go with it.

# Prediction on Test Set

In [40]:
X_train_merged = df_train['merged']
X_train_loc = df_train['loc_cleaned']

X_test_merged = df_test['merged']
X_test_loc = df_test['loc_cleaned']

y_train = df_train['salary']

In [41]:
tf1 = TfidfVectorizer(min_df=3, token_pattern=r'\w{3,}', ngram_range=(1,3))
tf2 = TfidfVectorizer(min_df=2, token_pattern=r'\w{3,}')

X_train_merged = tf1.fit_transform(X_train_merged)
X_train_loc = tf2.fit_transform(X_train_loc)

X_test_merged = tf1.transform(X_test_merged)
X_test_loc = tf2.transform(X_test_loc)

In [42]:
from scipy import sparse
from sklearn.preprocessing import StandardScaler

sc1 = StandardScaler()
X_train_MinExp = sc1.fit_transform(np.array(df_train['min_exp']).reshape(-1,1))
X_test_MinExp = sc1.transform(np.array(df_test['min_exp']).reshape(-1,1))
X_train_MinExp = sparse.csr_matrix(X_train_MinExp)
X_test_MinExp = sparse.csr_matrix(X_test_MinExp)

sc2 = StandardScaler()
X_train_MaxExp = sc2.fit_transform(np.array(df_train['max_exp']).reshape(-1,1))
X_test_MaxExp = sc2.transform(np.array(df_test['max_exp']).reshape(-1,1))
X_train_MaxExp = sparse.csr_matrix(X_train_MaxExp)
X_test_MaxExp = sparse.csr_matrix(X_test_MaxExp)

In [43]:
merged_train = hstack((X_train_merged, X_train_loc, X_train_MinExp, X_train_MaxExp))
merged_test = hstack((X_test_merged, X_test_loc, X_test_MinExp, X_test_MaxExp))

In [44]:
model = DecisionTreeClassifier(max_depth=5)

# Train the model
model.fit(merged_train, y_train)

# Predict on the training data (for evaluation)
train_predictions = model.predict(merged_train)

# Evaluate the model on the training data
print('Confusion Matrix\n', confusion_matrix(y_train, train_predictions))
print('Classification Report\n', classification_report(y_train, train_predictions))

print(f'Accuracy: {accuracy_score(y_train, train_predictions):.4f}')
print(f'Recall: {recall_score(y_train, train_predictions, average="weighted"):.4f}')
print(f'Precision: {precision_score(y_train, train_predictions, average="weighted"):.4f}')
print(f'F1 Score: {f1_score(y_train, train_predictions, average="weighted"):.4f}')
print(f'Cohen Kappa: {cohen_kappa_score(y_train, train_predictions):.4f}')

Confusion Matrix
 [[2375  230  101    2  462   75]
 [ 356 1771 1823  104  386   60]
 [ 145  787 2746  257  177   13]
 [  30  126  938  463   16    2]
 [ 863  573  292    5  884  207]
 [ 576 1206  802   32  601  315]]
Classification Report
               precision    recall  f1-score   support

        0to3       0.55      0.73      0.63      3245
      10to15       0.38      0.39      0.39      4500
      15to25       0.41      0.67      0.51      4125
      25to50       0.54      0.29      0.38      1575
        3to6       0.35      0.31      0.33      2824
       6to10       0.47      0.09      0.15      3532

    accuracy                           0.43     19801
   macro avg       0.45      0.41      0.40     19801
weighted avg       0.44      0.43      0.40     19801

Accuracy: 0.4320
Recall: 0.4320
Precision: 0.4369
F1 Score: 0.3999
Cohen Kappa: 0.3004


In [45]:
# Predict on the test data
test_predictions = model.predict(merged_test)

# Output the predictions
print("Predictions on test data:", test_predictions)


Predictions on test data: ['15to25' '0to3' '10to15' ... '15to25' '15to25' '10to15']


In [46]:
df_sub = pd.DataFrame(data=test_predictions, columns=['salary'])

In [47]:
!pip install xlsxwriter

In [ ]:
# Write the DataFrame to an Excel file
with pd.ExcelWriter('output.xlsx', engine='xlsxwriter') as writer:
    df_sub.to_excel(writer, sheet_name='Sheet1', index=False)

print("Predictions saved to output.xlsx")